# `presamples` tagged graph issue

## Problem
`bw2analyzer.recurse_tagged_database` doesn't work with LCA objects initialised with presamples

## Demo
### Import the necessary stuff

In [1]:
from brightway2 import *

In [2]:
import numpy as np

In [3]:
import presamples as ps

In [38]:
from bw2analyzer.tagged import recurse_tagged_database, aggregate_tagged_graph

### Set  up the brightway2 project

> Note - rather than run bw2setup() this takes a library project with ecoinvent 3.3 in it and copies it. 

> To recreate this notebook either do something similar or run bw2setup()

In [4]:
if 'presamples_test' in projects:
    projects.set_current('presamples_test')
else:
    projects.set_current('Ecoinvent3_3')
    projects.copy_project('presamples_test', switch=True)

### Create the simple system

Below is a picture of the system to be modelled. Basically it's set up so `input_1` has a known climate change impact of `1`, `input_2` has an impact of `2` and `input_3` has an impact of `3`. 

`a`, `b`, and `c` in the diagram are the exchange amounts of each input to `FU`. These all default to `1` (total impact of `6`). 

We're going to try and use `presamples` to run 3 different analyses where only `a`, `b` or `c` is set to `1` (others set to `0` sequentially, so the expected total impacts are `1`, `2` and `3` accordingly.

![](images/presamples_flow_chart.png)

In [5]:
CO2 = ('biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9')      # Carbon dioxide, fossil, kg

In [6]:
data = {
    ('test', 'FU'):{
        'exchanges': [{
            'input':('test', 'input_1'),
            'amount':1,
            'formula':'p_input_1',
            'type': 'technosphere'
        },{
            'input':('test', 'input_2'),
            'amount':1,
            'formula':'p_input_2',
            'type': 'technosphere'
        },{
            'input':('test', 'input_3'),
            'amount':1,
            'formula':'p_input_3',
            'type': 'technosphere'
        },
        ],
        'name': 'FU',
        'type': 'process'
    },
    ('test', 'input_1'):{
        'exchanges': [{
            'amount': 1,
            'input': CO2,
            'type': 'biosphere'
        }],
        'name': 'input_1',
        'type': 'process'
    },
    ('test', 'input_2'):{
        'exchanges': [{
            'amount': 2,
            'input': CO2,
            'type': 'biosphere'
        }],
        'name': 'input_2',
        'type': 'process'
    },
    ('test', 'input_3'):{
        'exchanges': [{
            'amount': 3,
            'input': CO2,
            'type': 'biosphere'
        }],
        'name': 'input_3',
        'type': 'process'
    }
}

### Create and write the database

In [8]:
db = Database("test")

In [9]:
db.write(data)

Writing activities to SQLite3 database:
0%  100%
[####] | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 05/30/2018 15:12:12
  Finished: 05/30/2018 15:12:12
  Total time elapsed: 00:00:00
  CPU %: 97.50
  Memory %: 0.81


### Define the functional unit and method

In [10]:
fu = db.search('FU')[0]

In [11]:
m = ('IPCC 2013', 'climate change', 'GTP 100a')

### Check the default lca score (expected = `6`)

In [12]:
lca = LCA({fu:1},m)

In [13]:
lca.lci()

In [14]:
lca.lcia()

In [15]:
lca.score

6.0

Ok that works - next to try it with `presamples`

### Generate the `presamples` package

#### Get a list of the inputs to FU

In [16]:
FU_inputs = [exc for exc in fu.technosphere()]

#### Figure out and store the matrix indices

In [17]:
FU_indices = [(exc['input'], exc['output'], 'technosphere')
                        for exc in FU_inputs
                       ]
FU_indices

[(('test', 'input_1'), ('test', 'FU'), 'technosphere'),
 (('test', 'input_2'), ('test', 'FU'), 'technosphere'),
 (('test', 'input_3'), ('test', 'FU'), 'technosphere')]

#### Set up the value matrix (3x3 identity matrix)

In [18]:
one_by_one = np.eye(3)

#### Prepare the `presamples` data

In [19]:
presamples_one_by_one = [(
    one_by_one,
    FU_indices,
    'technosphere'
)]

#### Create the `presamples` package

In [20]:
test_id, test_fp = ps.create_presamples_package(
    matrix_data=presamples_one_by_one,
    name='presamples_test_one_by_one',
    seed = 'sequential'
)

#### Test

- Set up the LCA
- Run the tagged graph (via `recurse_tagged_database` so we can keep our `LCA` object)
- Aggregate the tagged graph
- Sum the aggregated graph to get the total impact
- compare to `lca.score`

In [77]:
one_by_one_lca = LCA({fu:1}, m, presamples=[test_fp])

graphs = []

for i, k in enumerate(FU_inputs):

    if i==0:
        one_by_one_lca.lci()
        one_by_one_lca.lcia()
    else:
        one_by_one_lca.presamples.update_matrices(one_by_one_lca)
        one_by_one_lca.redo_lci()
        one_by_one_lca.redo_lcia()

    # Note: this is code excised from bw2analyzer.tagged.traverse_tagged_databases
    
    method_dict = {o[0]: o[1] for o in Method(m).load()}
    
    label = "name" # aggregate on name - i.e. traverse foreground model for contributions
    default_tag = "other"
    
    cfu = {fu:1} # change format of fu to include amount
    
    graph = [recurse_tagged_database(key, amount, method_dict, one_by_one_lca, label, default_tag)
                  for key, amount in cfu.items()]
    
    result = aggregate_tagged_graph(graph) # aggregate the tags
    
    graph_result = sum([v for k, v in result.items()]) # calculate the sum of the impacts
    
    # print the results
    
    print("\nRun {}: Expected result = {:.1f} kg-CO2e\n{}".format(i+1,i+1, "-"*44))
    print("Total impact from LCA score:\t{} kg-CO2e".format(one_by_one_lca.score))
    print("Total impact from graph:\t{} kg-CO2e\n{}".format(graph_result, "-"*44))
    
    graphs.append(graph) # create a list of the graphs


Run 1: Expected result = 1.0 kg-CO2e
--------------------------------------------
Total impact from LCA score:	1.0 kg-CO2e
Total impact from graph:	6.0 kg-CO2e
--------------------------------------------

Run 2: Expected result = 2.0 kg-CO2e
--------------------------------------------
Total impact from LCA score:	2.0 kg-CO2e
Total impact from graph:	6.0 kg-CO2e
--------------------------------------------

Run 3: Expected result = 3.0 kg-CO2e
--------------------------------------------
Total impact from LCA score:	3.0 kg-CO2e
Total impact from graph:	6.0 kg-CO2e
--------------------------------------------


### Result

`presamples` works as expected for LCA scores, but the tagged graph results reflect the original database

All of the tagged graphs are the same:

In [79]:
graphs[0] == graphs [1] == graphs[2]

True